In [ ]:
### THIS CELL SETS UP THE GOOGLE COLAB ENVIRONMENT. 
### IF RUNNING THIS NOTEBOOK LOCALLY, IT MAY BE SAFELY DELETED.

#@markdown Install the software by pressing the _Play_ button on the left.

try:
    import google.colab
    RUNNING_IN_COLAB = True
except ImportError:
    RUNNING_IN_COLAB = False
except Exception as e: 
    err = "Could not figure out if runnning in a colab notebook\n"
    raise Exception(err) from e

if RUNNING_IN_COLAB:
    !pip install dataprob


In [ ]:
%matplotlib inline
import numpy as np
import pandas as pd
import dataprob

def linear_model(m=0,b=0,x=None): 
    return m*x + b

# Observed data will have a slope of -5 and an intercept of 100
gen_params = {"m":-5,"b":100}

# Set up to collect 20 data points between -5 and 5, with an uncertainty on each 
# observation of 1
num_points = 20
x = np.linspace(-5,5,num_points)
err = np.random.normal(0,1,num_points)

# Generate y_obs and y_std. 
y_obs = linear_model(x=x,**gen_params) + err
y_std = 1

expt_df = pd.DataFrame({"y_obs":y_obs,
                        "y_std":y_std})

# we need to send `x` into our model as a non-fittable parameter holding the 
# x-values for the line. 
x = np.linspace(-5,5,num_points)
non_fit_kwargs = {"x":x}

# Generate Fitter. (Set method to "bootstrap" or "mcmc" to see the other
# analysis methods in action). 
f = dataprob.setup(some_function=linear_model,
                   method="ml",
                   non_fit_kwargs=non_fit_kwargs)

f.data_df = expt_df

# Run fit
f.fit()

fig = dataprob.plot_summary(f,
                            x_axis=x,
                            x_label="x",
                            y_label="y")

In [ ]:
fig = dataprob.plot_corner(f)

In [ ]:
# Show fit result dataframe
f.fit_df 

In [ ]:
f.fit_quality

### Compare input and fit parameters

In [ ]:
from matplotlib import pyplot as plt

fig, ax = plt.subplots(1,figsize=(6,6))

x = np.array(list(gen_params.values()))
y = np.array(f.fit_df.loc[gen_params.keys(),"estimate"])
yerr = np.array(f.fit_df.loc[gen_params.keys(),"std"])

combined = list(x)
combined.extend(y)
span = (np.max(combined) - np.min(combined))
extend_by = 0.1*span
offset = 0.03*span
min_value = np.min(combined) - extend_by
max_value = np.max(combined) + extend_by

ax.scatter(x,y,s=20,edgecolor='none',facecolor='red',zorder=1)
ax.errorbar(x=x,y=y,yerr=yerr,lw=0,elinewidth=1,capsize=3,color='black',zorder=2)

for i, k in enumerate(gen_params.keys()):
    ax.text(x=x[i] - offset,y=y[i] + offset,s=k)


ax.plot((min_value,max_value),(min_value,max_value),'--',color='gray',zorder=0)

ax.set_xlim(min_value,max_value)
ax.set_ylim(min_value,max_value)
ax.set_xlabel("input parameter value")
ax.set_ylabel("estimated parameter value")
